# 2. Production and Attraction Constrained Models



##### Origin Constrained

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [30]:
data = pd.read_csv('data.csv')
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

In [31]:
#create the formula (the "-1" indicates no intercept in the regression model).
data['log_jobs'] = np.log(data['jobs'] + 1)
formula = 'flows ~ station_origin + log_jobs + log_distance-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=data, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61413
Model:                            GLM   Df Residuals:                    61013
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.0168e+06
Date:                Wed, 24 Apr 2024   Deviance:                   1.8615e+06
Time:                        17:15:29   Pearson chi2:                 2.78e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

## 2.1 Model Estimates

In [32]:
O_i = pd.DataFrame(data.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub = data.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub = cdatasub.merge(D_j, on = "station_destination", how = "left" )

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/1695467687.py:1: FutureWarning: The provided callable <function sum at 0x103ae5700> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  O_i = pd.DataFrame(data.groupby(["station_origin"])["flows"].agg(np.sum))
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/1695467687.py:5: FutureWarning: The provided callable <function sum at 0x103ae5700> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))


In [33]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
#then once you have done this you can join them back into the dataframes
cdatasub = cdatasub.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
cdatasub.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub.head()

,station_origin,station_destination,flows,population,jobs,distance,unconstrainedEst1,log_population,log_jobs,log_distance,unconstrainedEst2,fitted,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,0,599.1,78549.1,8131.625097,10,6.395429,11.271492,9.003516,50,49.622673,599,78549,3.247926
1,Abbey Road,Beckton,1,599.1,442.1,8510.221774,0,6.395429,6.093795,9.049023,1,0.937341,599,442,3.247926
2,Abbey Road,Blackwall,3,599.1,665.1,3775.548872,0,6.395429,6.501440,8.236301,2,2.121621,599,665,3.247926
3,Abbey Road,Canary Wharf,1,599.1,58772.1,5086.614220,18,6.395429,10.981440,8.534368,53,53.302060,599,58772,3.247926
4,Abbey Road,Canning Town,37,599.1,15428.1,2229.023167,25,6.395429,9.644011,7.709319,32,32.210949,599,15428,3.247926


In [34]:
alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
beta = -prodSim.params[399]

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/3439944073.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = prodSim.params[398]
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/3439944073.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -prodSim.params[399]


In [35]:
alpha_i

station_origin[Abbey Road]          3.247926
station_origin[Acton Central]       5.014689
station_origin[Acton Town]          4.560696
station_origin[Aldgate]             3.321519
station_origin[Aldgate East]        3.455417
                                      ...   
station_origin[Wood Street]         5.239711
station_origin[Woodford]            5.158398
station_origin[Woodgrange Park]     5.252384
station_origin[Woodside Park]       4.696432
station_origin[Woolwich Arsenal]    6.903235
Length: 398, dtype: float64

In [36]:
gamma

0.7688636920138469

In [37]:
beta

0.8781494114824608

In [38]:
cdatasub["prodsimest1"] = np.exp(cdatasub["alpha_i"]+gamma*cdatasub["log_jobs"] - beta*cdatasub["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance,unconstrainedEst1,log_population,log_jobs,log_distance,unconstrainedEst2,fitted,O_i,D_j,alpha_i,prodsimest1
0,Abbey Road,Bank and Monument,0,599.1,78549.1,8131.625097,10,6.395429,11.271492,9.003516,50,49.622673,599,78549,3.247926,55.022080
1,Abbey Road,Beckton,1,599.1,442.1,8510.221774,0,6.395429,6.093795,9.049023,1,0.937341,599,442,3.247926,0.986916
2,Abbey Road,Blackwall,3,599.1,665.1,3775.548872,0,6.395429,6.501440,8.236301,2,2.121621,599,665,3.247926,2.756451
3,Abbey Road,Canary Wharf,1,599.1,58772.1,5086.614220,18,6.395429,10.981440,8.534368,53,53.302060,599,58772,3.247926,66.467040
4,Abbey Road,Canning Town,37,599.1,15428.1,2229.023167,25,6.395429,9.644011,7.709319,32,32.210949,599,15428,3.247926,49.054009
5,Abbey Road,Crossharbour,1,599.1,1208.1,6686.575560,0,6.395429,7.097632,8.807857,2,2.340318,599,1208,3.247926,2.639060
6,Abbey Road,Custom House,0,599.1,845.1,3824.955630,0,6.395429,6.740638,8.249302,3,2.525158,599,845,3.247926,3.275392
7,Abbey Road,Cutty Sark,2,599.1,1748.1,8503.998909,0,6.395429,7.466857,9.048292,3,2.668811,599,1748,3.247926,2.838197
8,Abbey Road,Cyprus,7,599.1,850.1,6532.199618,0,6.395429,6.746530,8.784499,2,1.817492,599,850,3.247926,2.056458
9,Abbey Road,Devons Road,1,599.1,611.1,3958.424171,0,6.395429,6.416896,8.283601,2,1.931506,599,611,3.247926,2.477883


### 2.2.1 The flow matrics

In [41]:
#first round the estimates
cdatasub["prodsimest1"] = round(cdatasub["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat3

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/608996508.py:4: FutureWarning: The provided callable <function sum at 0x103ae5700> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/608996508.py:4: FutureWarning: The provided callable <function sum at 0x103ae5700> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_11868/608996508.py:4: FutureWarning: The provided callable <fun

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,599.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1223.0
Acton Town,NaN,NaN,NaN,18.0,18.0,NaN,9.0,1.0,NaN,20.0,...,16.0,3.0,5.0,13.0,NaN,2.0,NaN,2.0,NaN,3749.0
Aldgate,NaN,NaN,2.0,NaN,47.0,NaN,NaN,0.0,NaN,21.0,...,4.0,NaN,3.0,2.0,NaN,1.0,NaN,1.0,NaN,2882.0
Aldgate East,NaN,NaN,2.0,52.0,NaN,NaN,1.0,0.0,NaN,23.0,...,5.0,1.0,3.0,2.0,NaN,1.0,NaN,1.0,NaN,3169.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,7.0,35.0,39.0,NaN,NaN,NaN,NaN,32.0,...,15.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,4867.0
Woodgrange Park,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532.0
Woodside Park,NaN,NaN,5.0,20.0,20.0,NaN,2.0,NaN,NaN,25.0,...,11.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3092.0


In [40]:
import scipy.stats
from math import sqrt
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [42]:
CalcRSquared(cdatasub["flows"], cdatasub["prodsimest1"])

0.3883341913949251

In [43]:
CalcRMSE(cdatasub["flows"], cdatasub["prodsimest1"])

102.888